Benchmark Scripts

In [1]:
import os
os.environ['PD_ON_RAY_CPU']  = str(4)

In [90]:
NUM_CPUS = 4
data_path = 'data/sample.pq'
use_pandas = False

In [4]:
import ray
ray.init(num_cpus=num_cpus)

import ray.dataframe as rdf

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:17381 to respond...
Waiting for redis server at 127.0.0.1:57495 to respond...
Starting local scheduler with the following resources: {'CPU': 4, 'GPU': 0}.

View the web UI at http://localhost:8889/notebooks/ray_ui80067.ipynb?token=4ffa8ecad83936c31d61cf8744a531f4db896817cee92c8a

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:30981 to respond...
Waiting for redis server at 127.0.0.1:50159 to respond...


In [38]:
import time
import json

In [89]:
import pandas as pd

In [91]:
if use_pandas:
    df = pd.read_parquet(data_path)
else:
    df = rdf.read_parquet(data_path)

In [57]:
def wait(df):
    if hasattr(df, "_block_partitions"):
        blocks = list(df._block_partitions.flatten())
        completed = ray.wait(blocks, num_returns=len(blocks)) #blocking

In [92]:
exprs = [
    # basic numeric
    "df.sum()",
    "df.sum(axis=1)",
    "df.median()",
    "df.median(axis=1)",
    "df.std()",
    "df.std(axis=1)",
    # transpose
    "wait(df.T)",
    # apply 
    "wait(df.apply(lambda x:x))",
    "df.apply('sum')",
    "df.apply('median')",
    "df.apply('std')",
    # sort
    "wait(df.sort_values(df.columns[0]))",
    "wait(df.sort_values(df.columns))",
    "df.idxmin()",
    # nunique
    "df.nunique()",
    "df.nunique(axis=1)"
]

In [93]:
def prof(expr):
    start = time.time()
    exec(expr)
    end = time.time()
    duration_sec = end - start
    return duration_sec

In [94]:
{expr:prof(expr) for expr in exprs}

{'df.sum()': 0.022571086883544922,
 'df.sum(axis=1)': 0.024512529373168945,
 'df.median()': 0.03245282173156738,
 'df.median(axis=1)': 0.030565500259399414,
 'df.std()': 0.0168912410736084,
 'df.std(axis=1)': 0.022594213485717773,
 'wait(df.T)': 0.018785953521728516,
 'wait(df.apply(lambda x:x))': 0.04767251014709473,
 "df.apply('sum')": 0.0192873477935791,
 "df.apply('median')": 0.017460346221923828,
 "df.apply('std')": 0.01680731773376465,
 'wait(df.sort_values(df.columns[0]))': 0.03315448760986328,
 'wait(df.sort_values(df.columns))': 0.04571247100830078,
 'df.idxmin()': 0.022420406341552734,
 'df.nunique()': 0.015996456146240234,
 'df.nunique(axis=1)': 0.04495549201965332}